# Initial AI Agent APPO w CNN (working) MR2 HS Disc

In [1]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [2]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import numpy as np
import gymnasium
from time import sleep

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 4.0
my_config["ep_max_length"] = 2000
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = False
my_config["benchmark"] = False
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'discreteAccel' : True,
  'accelAndBrake' : False,
  'discSteer' : True,
  'contAccelOnly' : False,
  'discAccelOnly' : False,
  'modelMode': 2,
  'agent' : 'APPO',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for APPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 1, # 1 is High Speed Ring, 2 is 0-400m, 
}

In [3]:
def env_creator(env_config):
    env = gymnasium.make("real-time-gym-v1", config=env_config)
    return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [4]:
import ray
ray.shutdown()
ray.init()

2023-10-07 16:37:26,122	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.13
Ray version:,2.6.3
Dashboard:,http://127.0.0.1:8265


In [5]:
from ray.rllib.algorithms.impala.impala import ImpalaConfig
algo = (
    ImpalaConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        batch_mode="truncate_episodes",
        )
    .framework(
        framework="torch",
        )
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
        train_batch_size=128,
        )
    # .model(
    #     post_fcnet_hiddens = [24,24],
    #     fcnet_activation = "relu",
    #     )
    .build()
)

#APPOConfig.framework("")

2023-10-07 16:37:28,918	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


ModuleNotFoundError: No module named 'ray.rllib.algorithms.APPO'

In [ ]:
N = 50000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()


(RolloutWorker pid=7720) reset triggered
(RolloutWorker pid=7720) reload save for track : 2


(RolloutWorker pid=7720) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch_tb\lib\site-packages\gymnasium\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
(RolloutWorker pid=7720)   if not isinstance(terminated, (bool, np.bool8)):


Loop:  0
Saved 0


(RolloutWorker pid=7720) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch_tb\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 4.227984499952072
(RolloutWorker pid=7720)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  1


...

(RolloutWorker pid=7720) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch_tb\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 3.985004199953437
(RolloutWorker pid=7720)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  72


(RolloutWorker pid=7720) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch_tb\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 3.936252499953298
(RolloutWorker pid=7720)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  73


(RolloutWorker pid=7720) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch_tb\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 4.1083025999532765
(RolloutWorker pid=7720)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
